In [1]:
import numpy as np

In [7]:
a = 1 / np.sqrt(0.9)

In [4]:
1 - 2 * 0.1469

0.7061999999999999

In [5]:
import scipy.stats as stats

In [6]:
X = stats.norm(0, 1)

In [8]:
X.cdf(a) - X.cdf(-a)

0.70815945485621157

In [9]:
y = [0.7, 0.3, 0.5]

$$
f(y_i | a) = \begin{cases}
a \cdot y_i^{a-1}, \text{ if } y_i \in [0;1] \\
0, \text{ otherwise } \\
\end{cases}
$$

Пишем функцию правдоподобия для отдельно взятого наблюдения!


In [14]:
def loglik(y, a):
    answer = np.log(a) + (a - 1) * np.log(y)
    return answer

Обязательно тестируем:

In [15]:
loglik(0.7, 3)

0.38526240079064489

Сейчас мы будем заниматься чёрной магией :)
Создадим класс `picasso`, который будет частным случаем класса `GenericLikelihoodModel`.

У класса `picasso` есть:
1. `__init__` — действия при создании модели
2. `fit` — действия при оценке модели
3. `nloglikeobs` — функция правдоподобия для отдельного наблюдения с противоположным знаком    
4. всё остальное будем само наследоваться, так как picasso — это частный случай GenericLikelihoodModel

In [35]:
from statsmodels.base.model import GenericLikelihoodModel
class picasso(GenericLikelihoodModel):
    def __init__(self, endog, exog=None, **kwds):
        if exog is None:
            exog = np.zeros_like(endog)
        super(picasso, self).__init__(endog, exog, **kwds)
    def nloglikeobs(self, params):
        m_loglik = -loglik(self.endog, params)
        return m_loglik
    def fit(self, start_params=None, maxiter=1000, maxfun=1000, **kwds):
        if start_params is None:
            start_params = 5
        return super(picasso, self).fit(start_params=start_params, 
                            maxiter=maxiter, maxfun=maxfun, **kwds)


In [33]:
y = [0.7, 0.3, 0.5]
problem_18_3 = picasso(y)

In [34]:
res = problem_18_3.fit()
res.summary()

Optimization terminated successfully.
         Current function value: -0.037262
         Iterations: 19
         Function evaluations: 38


<class 'statsmodels.iolib.summary.Summary'>
"""
                               picasso Results                                
==============================================================================
Dep. Variable:                      y   Log-Likelihood:                0.11179
Model:                        picasso   AIC:                           -0.2236
Method:            Maximum Likelihood   BIC:                           -0.2236
Date:                Mon, 06 Mar 2017                                         
Time:                        17:21:14                                         
No. Observations:                   3                                         
Df Residuals:                       3                                         
Df Model:                          -1                                         
==============================================================================
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const          1.3311      0.769      1.732      0.083        -0.175     2.837
==============================================================================
"""